In [1]:
!pip install nltk

In [2]:
!pip install spacy

In [256]:
import nltk
import spacy
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import json

from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Masking, Reshape, TimeDistributed, Bidirectional, Dropout, Layer, GlobalAveragePooling1D
from tensorflow.keras.models import Model, clone_model, Sequential

In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [234]:
MAX_NUMBER_OF_WORDS_PER_SENTENCE = 45
MAX_NUMBER_OF_SENTENCES = 10
MAX_NUMBER_OF_LETTERS = 550
VOCAB_SIZE = 800
NUM_EPOCHS = 100
BATCH_SIZE = 32
DROPOUT_RATE = 0.2
LSTM_SIZE = 256
EMBEDDING_DIM = 256

In [6]:
dataset_csv = "data.csv"

In [7]:
data_frame = pd.read_csv(dataset_csv)
data_frame

,Problem,Difficulty,Tags,Worst Complexity,Link
0,Given an array of integers nums and an integer...,Easy,Brute force;Hashmap,N^2,https://leetcode.com/problems/two-sum/
1,You are given two non-empty linked lists repre...,Medium,Linked List;Implementation,N,https://leetcode.com/problems/add-two-numbers/
2,"Given a string s, find the length of the longe...",Medium,Sliding Window;Hashmap,N,https://leetcode.com/problems/longest-substrin...
3,Given two sorted arrays nums1 and nums2 of siz...,Hard,Binary Search;Divide and conquer,log(N),https://leetcode.com/problems/median-of-two-so...
4,"Given a string s, return the longest \npalindr...",Medium,Dynamic Programming,N^2,https://leetcode.com/problems/longest-palindro...
...,...,...,...,...,...
68,"Given an m x n integers matrix, return the len...",Hard,Dynamic Programming,N^2,https://leetcode.com/problems/longest-increasi...
69,Write a function that reverses a string. The i...,Easy,Implementation,N,https://leetcode.com/problems/reverse-string/
70,"Given an integer array nums and an integer k, ...",Medium,Hashmap,N,https://leetcode.com/problems/top-k-frequent-e...
71,"Given a string s, find the first non-repeating...",Easy,Hashmap,N,https://leetcode.com/problems/first-unique-cha...


In [8]:
def clean_text(text):
  text = text.strip()
  text = text.lower()

  lemmatizer = WordNetLemmatizer()
  words = text.split()
  words = [lemmatizer.lemmatize(word) for word in words]
  text = ' '.join(words)

  return text

nlp = spacy.load("en_core_web_sm")
def split_text_to_sentences(text):
    doc = nlp(text)

    sentences = [sentence.text for sentence in doc.sents]

    return sentences



In [9]:
def clean_dataframe(data_frame):
    data_frame["Problem"] = data_frame["Problem"].apply(lambda problem: clean_text(problem))
    return data_frame

In [10]:
def split_dataframe_by_tag(data_frame):
    data_frame = data_frame.assign(Tag=data_frame['Tags'].str.split(';')).explode('Tag')
    return data_frame

In [11]:
data_frame = clean_dataframe(data_frame)


In [180]:
processed_dataframe, test_dataframe = train_test_split(data_frame, test_size = 0.4, random_state = 42)

In [181]:
test_dataframe

,Problem,Difficulty,Tags,Worst Complexity,Link
4,"given a string s, return the longest palindrom...",Medium,Dynamic Programming,N^2,https://leetcode.com/problems/longest-palindro...
63,given an array nums containing n distinct numb...,Easy,Bit manipulation,N,https://leetcode.com/problems/missing-number/
18,"given an array nums of distinct integers, retu...",Medium,Backtracking,N!,https://leetcode.com/problems/permutations/
0,given an array of integer nums and an integer ...,Easy,Brute force;Hashmap,N^2,https://leetcode.com/problems/two-sum/
28,given an m x n grid of character board and a s...,Medium,DFS,N^2,https://leetcode.com/problems/word-search/
72,"given four integer array nums1, nums2, nums3, ...",Medium,Hashmap;Binary Search,N^2,https://leetcode.com/problems/4sum-ii/descript...
10,given a string s containing just the character...,Easy,Stack,N,https://leetcode.com/problems/valid-parentheses/
34,you are given an integer array price where pri...,Medium,Dynamic Programming,N,https://leetcode.com/problems/best-time-to-buy...
12,you are given the head of two sorted linked li...,Easy,Linked List;Implementation,N,https://leetcode.com/problems/merge-two-sorted...
55,"given the head of a singly linked list, revers...",Easy,Implementation,N,https://leetcode.com/problems/reverse-linked-l...


In [182]:
processed_dataframe = split_dataframe_by_tag(processed_dataframe)

In [183]:
processed_dataframe

,Problem,Difficulty,Tags,Worst Complexity,Link,Tag
49,"given an array nums of size n, return the majo...",Easy,Implementation,1,https://leetcode.com/problems/majority-element/,Implementation
19,return k.you are given an n x n 2d matrix repr...,Medium,Implementation,N^2,https://leetcode.com/problems/rotate-image/,Implementation
59,"given the root of a binary search tree, and an...",Medium,BFS;DFS,N,https://leetcode.com/problems/kth-smallest-ele...,BFS
59,"given the root of a binary search tree, and an...",Medium,BFS;DFS,N,https://leetcode.com/problems/kth-smallest-ele...,DFS
25,"given an array nums with n object colored red,...",Medium,Sliding Window,N,https://leetcode.com/problems/sort-colors/solu...,Sliding Window
40,every si for 1 <= i <= k is in wordlist. note ...,Medium,Dynamic Programming;Backtracking,N*2^N,https://leetcode.com/problems/palindrome-parti...,Dynamic Programming
40,every si for 1 <= i <= k is in wordlist. note ...,Medium,Dynamic Programming;Backtracking,N*2^N,https://leetcode.com/problems/palindrome-parti...,Backtracking
13,you are given an array of k linked-lists lists...,Hard,Priority Queue;Merge sort,NlogN,https://leetcode.com/problems/merge-k-sorted-l...,Priority Queue
13,you are given an array of k linked-lists lists...,Hard,Priority Queue;Merge sort,NlogN,https://leetcode.com/problems/merge-k-sorted-l...,Merge sort
53,you are a professional robber planning to rob ...,Medium,Dynamic Programming,N,https://leetcode.com/problems/house-robber/,Dynamic Programming


In [16]:
def analyze_dataframe(data_frame):
  max_number_of_letters = 0
  max_number_of_words_per_sentence = 0
  max_number_of_sentences = 0
  vocabs = dict()
  problems_count_by_tag = defaultdict(int)
  problems_count_by_difficulty = defaultdict(int)
  problems_count_by_time_complexity = defaultdict(int)

  for row in processed_dataframe.iterrows():
      row_content = row[1]
      problem, tag, time_complexity, difficulty = row_content["Problem"], row_content["Tag"], row_content["Worst Complexity"], row_content["Difficulty"]

      max_number_of_letters = max(max_number_of_letters, len(problem))

      problem_sentences = split_text_to_sentences(problem)
      number_of_sentences = len(problem_sentences)
      max_number_of_sentences = max(max_number_of_sentences, number_of_sentences)

      for sentence in problem_sentences:
        words = sentence.split(" ")

        for word in words:
          vocabs[word] = 1

        number_of_words = len(words)
        max_number_of_words_per_sentence = max(max_number_of_words_per_sentence, number_of_words)

        problems_count_by_tag[tag] += 1
        problems_count_by_difficulty[difficulty] += 1
        problems_count_by_time_complexity[time_complexity] += 1

  return {
      "max_number_of_words_per_sentence": max_number_of_words_per_sentence,
      "max_number_of_sentences": max_number_of_sentences,
      "max_number_of_letters": max_number_of_letters,
      "vocabs": vocabs,
      "problems_count_by_tag": problems_count_by_tag,
      "problems_count_by_difficulty": problems_count_by_difficulty,
      "problems_count_by_time_complexity": problems_count_by_time_complexity
  }





In [17]:
# Only run this when you want to analyze the dataframe
# analyze_result = analyze_dataframe(processed_dataframe)

In [184]:
def get_tokenizer(texts):
    tokenizer = Tokenizer(num_words = VOCAB_SIZE)
    tokenizer.fit_on_texts(texts)
    return tokenizer

In [185]:
problems = processed_dataframe["Problem"]
tokenizer = get_tokenizer(problems)

In [186]:
tokenizer

In [187]:
def permute_sentences(text):
    sentences = split_text_to_sentences(text)
    random.shuffle(sentences)  # Shuffle the sentences randomly
    permuted_text = '. '.join(sentences)  # Join the sentences back into a text string
    return permuted_text


In [188]:
# Permute the sentences in the problem to get a little more data
def permute_problem_sentences_in_dataframe(data_frame):
    new_rows = []
    for row in data_frame.iterrows():
        row_content = row[1]
        problem, tag, time_complexity, difficulty, link = row_content["Problem"], row_content["Tag"], row_content["Worst Complexity"], \
            row_content["Difficulty"], row_content["Link"]
        permuted_problem = permute_sentences(problem)
        new_row = {
            "Problem": permuted_problem,
            "Tag": tag,
            "Worst Complexity": time_complexity,
            "Difficulty": difficulty,
            "Link": link
        }
        new_rows.append(new_row)

    new_data_frame = data_frame.append(new_rows)
    return new_data_frame


In [189]:
processed_dataframe = permute_problem_sentences_in_dataframe(processed_dataframe)

<ipython-input-188-7edc55b3d457>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data_frame = data_frame.append(new_rows)


In [190]:
processed_dataframe

,Problem,Difficulty,Tags,Worst Complexity,Link,Tag
49,"given an array nums of size n, return the majo...",Easy,Implementation,1,https://leetcode.com/problems/majority-element/,Implementation
19,return k.you are given an n x n 2d matrix repr...,Medium,Implementation,N^2,https://leetcode.com/problems/rotate-image/,Implementation
59,"given the root of a binary search tree, and an...",Medium,BFS;DFS,N,https://leetcode.com/problems/kth-smallest-ele...,BFS
59,"given the root of a binary search tree, and an...",Medium,BFS;DFS,N,https://leetcode.com/problems/kth-smallest-ele...,DFS
25,"given an array nums with n object colored red,...",Medium,Sliding Window,N,https://leetcode.com/problems/sort-colors/solu...,Sliding Window
...,...,...,...,...,...,...
49,the robot can only move either down or right a...,Medium,NaN,N^2,https://leetcode.com/problems/unique-paths/,Dynamic Programming
50,"given an integer array nums, find the subarray...",Medium,NaN,N,https://leetcode.com/problems/maximum-subarray/,Dynamic Programming
51,the product of any prefix or suffix of nums is...,Medium,NaN,N,https://leetcode.com/problems/product-of-array...,Dynamic Programming
52,"given two string needle and haystack, return t...",Easy,NaN,N^2,https://leetcode.com/problems/find-the-index-o...,Sliding Window


In [191]:
def encode_dataframe_column(data_frame, column_name):
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(data_frame[column_name])
    type_mapping = {label: i for i, label in enumerate(label_encoder.classes_)}

    return {
        "type_mapping": type_mapping,
        "encoded_labels": encoded_labels
    }

In [192]:
tag_encoded_info = encode_dataframe_column(processed_dataframe, "Tag")
processed_dataframe["Tag"] = tag_encoded_info["encoded_labels"]

In [193]:
tag_encoded_info

{'type_mapping': {'BFS': 0,
  'Backtracking': 1,
  'Binary Search': 2,
  'Bit manipulation': 3,
  'DFS': 4,
  'Divide and conquer': 5,
  'Dynamic Programming': 6,
  'Greedy': 7,
  'Hashmap': 8,
  'Implementation': 9,
  'Linked List': 10,
  'Math': 11,
  'Merge sort': 12,
  'Priority Queue': 13,
  'Sliding Window': 14,
  'Sorting': 15,
  'Trie': 16},
 'encoded_labels': array([ 9,  9,  0,  4, 14,  6,  1, 13, 12,  6,  2,  5,  9,  6,  1,  8, 16,
        15,  9, 10,  9,  0,  4,  8, 11,  8,  2,  1,  7, 14,  2,  6,  8,  6,
         1,  4,  9,  3,  0,  4,  9,  6,  6, 10,  9,  3,  6, 14,  8,  6,  6,
         6, 14,  3,  9,  9,  0,  4, 14,  6,  1, 13, 12,  6,  2,  5,  9,  6,
         1,  8, 16, 15,  9, 10,  9,  0,  4,  8, 11,  8,  2,  1,  7, 14,  2,
         6,  8,  6,  1,  4,  9,  3,  0,  4,  9,  6,  6, 10,  9,  3,  6, 14,
         8,  6,  6,  6, 14,  3])}

In [194]:
complexity_encoded_info = encode_dataframe_column(processed_dataframe, "Worst Complexity")
processed_dataframe["Worst Complexity"] = complexity_encoded_info["encoded_labels"]


In [195]:
complexity_encoded_info

{'type_mapping': {'1': 0,
  '2^N': 1,
  '4^N': 2,
  'N': 3,
  'N*2^N': 4,
  'N^2': 5,
  'N^3': 6,
  'NlogN': 7,
  'log(N)': 8},
 'encoded_labels': array([0, 5, 3, 3, 3, 4, 4, 7, 7, 3, 8, 8, 3, 3, 2, 3, 5, 5, 3, 3, 3, 5,
        5, 5, 5, 3, 8, 1, 3, 3, 8, 3, 3, 3, 2, 3, 3, 3, 5, 5, 3, 6, 5, 3,
        3, 0, 3, 3, 3, 5, 3, 3, 5, 0, 0, 5, 3, 3, 3, 4, 4, 7, 7, 3, 8, 8,
        3, 3, 2, 3, 5, 5, 3, 3, 3, 5, 5, 5, 5, 3, 8, 1, 3, 3, 8, 3, 3, 3,
        2, 3, 3, 3, 5, 5, 3, 6, 5, 3, 3, 0, 3, 3, 3, 5, 3, 3, 5, 0])}

In [196]:
difficulty_encoded_info = encode_dataframe_column(processed_dataframe, "Difficulty")
processed_dataframe["Difficulty"] = difficulty_encoded_info["encoded_labels"]

In [197]:
difficulty_encoded_info

{'type_mapping': {'Easy': 0, 'Hard': 1, 'Medium': 2},
 'encoded_labels': array([0, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 2, 0, 0, 0, 0, 2, 2, 2,
        2, 1, 1, 0, 2, 2, 2, 1, 2, 0, 0, 2, 2, 0, 0, 2, 1, 1, 0, 2, 1, 2,
        2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1,
        1, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 1, 1, 0, 2, 2, 2, 1, 2, 0, 0, 2,
        2, 0, 0, 2, 1, 1, 0, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0])}

In [198]:
problems = processed_dataframe["Problem"].tolist()
complexity = processed_dataframe["Worst Complexity"]
difficulty = processed_dataframe["Difficulty"]
tag = processed_dataframe["Tag"]

In [199]:
def tokenize_text(text, tokenizer):
    tokenized_text = tokenizer.texts_to_sequences([text])[0]
    tokenized_text = pad_sequences([tokenized_text], maxlen=MAX_NUMBER_OF_LETTERS, padding='post', truncating='post')[0]
    return tokenized_text.tolist()

In [115]:
test_problem = problems[10]
tokenized_problem = tokenize_text(test_problem, tokenizer)

In [251]:
def get_model(num_types, embedding_dim = EMBEDDING_DIM):
    model = Sequential()
    model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=embedding_dim))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(units=num_types, activation='softmax'))
    return model

In [241]:
def convert_single_label_to_vector(label_num, types):
  label_vector = [0 for _ in range(len(types))]
  label_vector[label_num] = 1

  return label_vector

In [250]:
def train_model(model, train_texts, train_labels, num_epochs = NUM_EPOCHS, batch_size = BATCH_SIZE):
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(train_texts, train_labels, epochs=num_epochs, batch_size=batch_size)

In [203]:
difficulty_type_mapping = difficulty_encoded_info["type_mapping"]

In [252]:
difficulty_model = get_model(len(difficulty_type_mapping))

In [243]:
difficulty_model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, None, 256)         204800    
                                                                 
 global_average_pooling1d_10  (None, 256)              0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_18 (Dense)            (None, 3)                 771       
                                                                 
Total params: 205,571
Trainable params: 205,571
Non-trainable params: 0
_________________________________________________________________


In [244]:
difficulty_labels = [convert_single_label_to_vector(label, difficulty_type_mapping.keys()) for label in difficulty]

In [245]:
tokenized_problems = [tokenize_text(problem, tokenizer) for problem in problems]

In [253]:
train_model(difficulty_model, tokenized_problems, difficulty_labels)

Epoch 1/100
4/4 [==============================] - 1s 30ms/step - loss: 1.1148 - accuracy: 0.2593
Epoch 2/100
4/4 [==============================] - 0s 19ms/step - loss: 1.0866 - accuracy: 0.5370
Epoch 3/100
4/4 [==============================] - 0s 16ms/step - loss: 1.0616 - accuracy: 0.5370
Epoch 4/100
4/4 [==============================] - 0s 16ms/step - loss: 1.0427 - accuracy: 0.5370
Epoch 5/100
4/4 [==============================] - 0s 17ms/step - loss: 1.0297 - accuracy: 0.5370
Epoch 6/100
4/4 [==============================] - 0s 16ms/step - loss: 1.0198 - accuracy: 0.5370
Epoch 7/100
4/4 [==============================] - 0s 16ms/step - loss: 1.0123 - accuracy: 0.5370
Epoch 8/100
4/4 [==============================] - 0s 23ms/step - loss: 1.0067 - accuracy: 0.5370
Epoch 9/100
4/4 [==============================] - 0s 17ms/step - loss: 1.0050 - accuracy: 0.5370
Epoch 10/100
4/4 [==============================] - 0s 17ms/step - loss: 1.0029 - accuracy: 0.5370
Epoch 11/100
4/4 [=

In [247]:
def predict_text(model, types_list, tokenizer, text):
    tokenized_text = tokenize_text(text, tokenizer)
    probs = model.predict([tokenized_text])[0]
    result = {types_list[index]: probs[index] for index in range(len(types_list))}
    return result

In [254]:
# Evaludate difficulty model
correct_labels = test_dataframe["Difficulty"].tolist()
number_of_correct_predictions = 0
index = 0

for problem in test_dataframe["Problem"]:
    probs = predict_text( difficulty_model, list(difficulty_type_mapping.keys()), tokenizer, problem)
    highest_prob = 0
    label = "Easy"
    for key in probs.keys():
        prob = probs[key]
        if prob > highest_prob:
            highest_prob = prob
            label = key

    correct_label = correct_labels[index]
    if label == correct_label:
        number_of_correct_predictions += 1


    index += 1

print(number_of_correct_predictions / len(correct_labels))

1/1 [==============================] - 0s 58ms/step
0.6


In [255]:
difficulty_model.save("difficulty_model.h5")

In [263]:
complexity_type_mapping = complexity_encoded_info["type_mapping"]
complexity_model = get_model(len(complexity_type_mapping))
complexity_labels = [convert_single_label_to_vector(label, complexity_type_mapping.keys()) for label in complexity]
train_model(complexity_model, tokenized_problems, complexity_labels, num_epochs = 250)

Epoch 1/250
4/4 [==============================] - 1s 46ms/step - loss: 2.1644 - accuracy: 0.5000
Epoch 2/250
4/4 [==============================] - 0s 25ms/step - loss: 2.1095 - accuracy: 0.5000
Epoch 3/250
4/4 [==============================] - 0s 23ms/step - loss: 2.0573 - accuracy: 0.5000
Epoch 4/250
4/4 [==============================] - 0s 24ms/step - loss: 2.0039 - accuracy: 0.5000
Epoch 5/250
4/4 [==============================] - 0s 26ms/step - loss: 1.9500 - accuracy: 0.5000
Epoch 6/250
4/4 [==============================] - 0s 22ms/step - loss: 1.8989 - accuracy: 0.5000
Epoch 7/250
4/4 [==============================] - 0s 25ms/step - loss: 1.8490 - accuracy: 0.5000
Epoch 8/250
4/4 [==============================] - 0s 22ms/step - loss: 1.7965 - accuracy: 0.5000
Epoch 9/250
4/4 [==============================] - 0s 24ms/step - loss: 1.7469 - accuracy: 0.5000
Epoch 10/250
4/4 [==============================] - 0s 26ms/step - loss: 1.7019 - accuracy: 0.5000
Epoch 11/250
4/4 [=

In [265]:
# Evaludate complexity model
correct_labels = test_dataframe["Worst Complexity"].tolist()
number_of_correct_predictions = 0
index = 0

for problem in test_dataframe["Problem"]:
    probs = predict_text( complexity_model, list(complexity_type_mapping.keys()), tokenizer, problem)
    highest_prob = 0
    label = None
    for key in probs.keys():
        prob = probs[key]
        if prob > highest_prob:
            highest_prob = prob
            label = key

    correct_label = correct_labels[index]
    if label == correct_label:
        number_of_correct_predictions += 1

    print(label, correct_label)

    index += 1

print(number_of_correct_predictions / len(correct_labels))

1/1 [==============================] - 0s 76ms/step
N N^2
1/1 [==============================] - 0s 66ms/step
N N
1/1 [==============================] - 0s 62ms/step
N N!
1/1 [==============================] - 0s 48ms/step
N N^2
1/1 [==============================] - 0s 43ms/step
N^2 N^2
1/1 [==============================] - 0s 42ms/step
N N^2
1/1 [==============================] - 0s 49ms/step
N N
1/1 [==============================] - 0s 42ms/step
N N
1/1 [==============================] - 0s 48ms/step
N N
1/1 [==============================] - 0s 46ms/step
N N
1/1 [==============================] - 0s 51ms/step
N N^2
1/1 [==============================] - 0s 46ms/step
N N
1/1 [==============================] - 0s 47ms/step
N N
1/1 [==============================] - 0s 45ms/step
N N
1/1 [==============================] - 0s 47ms/step
N N
1/1 [==============================] - 0s 64ms/step
N NlogN
1/1 [==============================] - 0s 48ms/step
N NlogN
1/1 [======================

In [266]:
complexity_model.save("complexity_model.h5")

In [270]:
tag_type_mapping = tag_encoded_info["type_mapping"]
tag_model = get_model(len(tag_type_mapping))
tag_labels = [convert_single_label_to_vector(label, tag_type_mapping.keys()) for label in tag]
train_model(tag_model, tokenized_problems, tag_labels, num_epochs = 500)

Epoch 1/500
4/4 [==============================] - 1s 36ms/step - loss: 2.8134 - accuracy: 0.0741
Epoch 2/500
4/4 [==============================] - 0s 32ms/step - loss: 2.7878 - accuracy: 0.1759
Epoch 3/500
4/4 [==============================] - 0s 37ms/step - loss: 2.7645 - accuracy: 0.2037
Epoch 4/500
4/4 [==============================] - 0s 30ms/step - loss: 2.7408 - accuracy: 0.2037
Epoch 5/500
4/4 [==============================] - 0s 29ms/step - loss: 2.7161 - accuracy: 0.2037
Epoch 6/500
4/4 [==============================] - 0s 30ms/step - loss: 2.6969 - accuracy: 0.2037
Epoch 7/500
4/4 [==============================] - 0s 27ms/step - loss: 2.6731 - accuracy: 0.2037
Epoch 8/500
4/4 [==============================] - 0s 26ms/step - loss: 2.6499 - accuracy: 0.2037
Epoch 9/500
4/4 [==============================] - 0s 17ms/step - loss: 2.6295 - accuracy: 0.2037
Epoch 10/500
4/4 [==============================] - 0s 16ms/step - loss: 2.6119 - accuracy: 0.2037
Epoch 11/500
4/4 [=

In [278]:
# Evaludate tag model
correct_labels = test_dataframe["Tags"].tolist()
number_of_correct_predictions = 0
index = 0

for problem in test_dataframe["Problem"]:
    probs = predict_text( tag_model, list(tag_type_mapping.keys()), tokenizer, problem)
    probs_list = [(probs[label], label) for label in probs.keys()]
    probs_list = sorted(probs_list, key = lambda item: -item[0])
    top_probs_list = probs_list[0:5]

    correct_label = correct_labels[index]
    correct_prediction = 0
    for prob_tuple in top_probs_list:
        prob, label = prob_tuple
        if label in correct_label:
            correct_prediction = 1

    number_of_correct_predictions += correct_prediction
    index += 1


    # highest_prob = 0
    # label = None
    # for key in probs.keys():
    #     prob = probs[key]
    #     if prob > highest_prob:
    #         highest_prob = prob
    #         label = key

    # correct_label = correct_labels[index]

    # if label in correct_label:
    #     number_of_correct_predictions += 1

    # index += 1

print(number_of_correct_predictions / len(correct_labels))

1/1 [==============================] - 0s 53ms/step
0.8333333333333334


In [279]:
tag_model.save("tag_model.h5")